This notebook aims to recreate a simulator for Fujitsu's digital annealing algorithm that runs on a CPU and/or GPU. <br>
Ref. arXiv:1806.08815

In [1]:
import numpy as np
import numba as nb
import time

In [2]:
def default_temp_schedule(num_iter, temp_start, decay_rate, mode='EXPONENTIAL'):
    """
    Generates a list of temperatures for annealing algorithms.
    
    Parameters:
        num_iter (int): Length of the list.
        temp_start (number): Value of the first element in the returned list.
        decay_rate (number): Multiplier for changing the temperature during annealing.
        mode (string, default='EXPONENTIAL'):
            Three modes are possible. Note the accepted ranges for decay_rate are different.
            'EXPONENTIAL':  T[i+1] = T[i] * (1 - decay_rate)           # 0 <= decay_rate < 1
            'INVERSE':      T[i+1] = T[i] * (1 - decay_rate * T[i])    # 0 <= decay_rate < 1/temp_start
            'INVERSE_ROOT': T[i+1] = T[i] * (1 - decay_rate * T[i]**2) # 0 <= decay_rate < 1/temp_start**2
    
    Return: temp_schedule (list[number])
    """
    
    if mode == 'EXPONENTIAL':
        if 0 <= decay_rate < 1:
            TS = [temp_start]
            for _ in range(num_iter - 1):
                TS.append(TS[-1] * (1 - decay_rate))
            return TS
        else:
            raise ValueError("decay_rate out of accepted range")
    elif mode == 'INVERSE':
        if 0 <= decay_rate < 1/temp_start:
            TS = [temp_start]
            for _ in range(num_iter - 1):
                TS.append(TS[-1] * (1 - decay_rate * TS[-1]))
            return TS
        else:
            raise ValueError("decay_rate out of accepted range")
    elif mode == 'INVERSE_ROOT':
        if 0 <= decay_rate < 1/temp_start**2:
            TS = [temp_start]
            for _ in range(num_iter - 1):
                TS.append(TS[-1] * (1 - decay_rate * TS[-1]**2))
            return TS
        else:
            raise ValueError("decay_rate out of accepted range")
    else:
        raise ValueError("mode not supported")

In [12]:
@nb.njit(parallel=False)
def one_DA_run(Q_matrix, temp_schedule, ansatz_state=None, offset_increase_rate=0.):
    """
    One digital annealing run over the full temperature schedule.
    
    Parameters:
        Q_matrix (2-D array of float64): The matrix representing the local and coupling field of the problem.
        temp_schedule (list[float64]): The annealing temperature schedule.
                                       The number of iterations is implicitly the length of temp_schedule.
        ansatz_state (1-D array of bool, default=None): The boolean vector representing the initial state.
                                                        If None, a random state is chosen.
        offset_increase_rate (float64, default=0): The parameter that prevents from being in the same state for too long.
    
    Return: final_state (1-D array of bool)
    """
    
    # Q_coef[i][j]: local field of i if i==j; coupling strength if i!=j
    Q_coef = Q_matrix + Q_matrix.T - np.diag(Q_matrix)
    N = Q_matrix.shape[0]
    E_offset = 0.
    
    if ansatz_state is None:
        state = (np.random.binomial(1, 0.5, N) == 1)
    else:
        state = ansatz_state
    
    for temp in temp_schedule:
        accepted = np.zeros(N, dtype=np.bool_)
        for flip in nb.prange(N):
            delta_E = (1 - 2*state[flip]) * (np.sum(Q_coef[flip][state]) + Q_coef[flip, flip] * (1 - state[flip]))
            if np.random.binomial(1, np.minimum(np.exp(-(delta_E - E_offset)/temp), 1.)):
                accepted[flip] = True
        
        if np.any(accepted): # at least one flip is accepted
            # a random bit flip is chosen from all the accepted flips
            state[np.random.choice(accepted.nonzero()[0])] ^= True
            E_offset = 0.
        else:
            E_offset += offset_increase_rate
    
    return state

In [4]:
Q = np.array([[-1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]])
ansatz = np.zeros(4, dtype=np.bool_)
TS = default_temp_schedule(10000, 300., 0.001)

In [13]:
# Without numba
np.random.seed(0)
start_time = time.time()
ans = one_DA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [ True False False False]; time: 1.2739269733428955 s


In [7]:
# With numba, not parallelized, first pass
np.random.seed(0)
start_time = time.time()
ans = one_DA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

C:\ProgramData\Anaconda3\lib\site-packages\numba\core\ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'temp_schedule' of function 'one_DA_run'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-6-6ead41ea293d>", line 2:
@nb.njit(parallel=False)
def one_DA_run(Q_matrix, temp_schedule, ansatz_state=None, offset_increase_rate=0.):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


ground state: [ True False False False]; time: 2.25362229347229 s


In [8]:
# With numba, not parallelized, second pass
np.random.seed(0)
start_time = time.time()
ans = one_DA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [ True False False False]; time: 0.03602242469787598 s


In [10]:
# With numba, parallelized, first pass
np.random.seed(0)
start_time = time.time()
ans = one_DA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

C:\ProgramData\Anaconda3\lib\site-packages\numba\core\ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'temp_schedule' of function 'one_DA_run'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-9-711b639fac8b>", line 2:
@nb.njit(parallel=True)
def one_DA_run(Q_matrix, temp_schedule, ansatz_state=None, offset_increase_rate=0.):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


ground state: [ True False False False]; time: 3.3223562240600586 s


In [11]:
# With numba, parallelized, second pass
np.random.seed(0)
start_time = time.time()
ans = one_DA_run(Q, TS, ansatz_state=ansatz)
total_time = time.time() - start_time
print(f'ground state: {ans}; time: {total_time} s')

ground state: [ True False False False]; time: 0.09706830978393555 s
